# Project Title

## Introduction

In this section, briefly describe:
- **Goal of the analysis** — What are you trying to achieve?
- **Key question(s)** — The main problem(s) or topic(s) you aim to answer.
- **Context or background** — Why this question matters, and any relevant domain info.

## Table of Contents
1. [Overview](#Introduction)

2. [Setup & Scope](#setup--scope)

3. [Data](#data)

4. [Analysis](#Analysis)

5. [Conclusions](#Conclusions)

## Setup & Scope 

- **Data source(s)** — Where the data comes from and how it was obtained.
- **Scope and boundaries** — What’s included, what’s out of scope, and any assumptions.
- **Success criteria** — How you’ll judge whether the analysis answers the question.

*Keep this concise — 3 to 6 sentences is plenty.*

In [2]:
# Data handling
import numpy as np
import pandas as pd

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns


## Data

Describe: The dataset from Codecademy is inspired by information from the National Parks Service on endangered species across various parks.

- **Data source(s)** — two data files based on 
- **Data description** — Number of rows/columns, variable types, key fields.
- **Data dictionary** — What each column means.
- **Quality notes** — Missing values, potential biases, limitations.
- **Pre‑processing steps** — Cleaning, formatting, feature engineering.



In [19]:
observations = pd.read_csv('observations.csv')
species = pd.read_csv('species_info.csv')
# species.info()
data = pd.merge(observations, species, on='scientific_name', how='left')
data.head()
data.isnull().sum()  # Check for missing values in the 'sum' column


scientific_name            0
park_name                  0
observations               0
category                   0
common_names               0
conservation_status    24752
dtype: int64

## Analysis

In this combined section, cover:
- **Approach** — The methods you used to explore and answer the question.
- **Exploratory findings** — Summary stats, plots, correlations.
- **Techniques applied** — Statistical tests, models, transformations, etc.
- **Results** — Tables, charts, metrics.
- **Interpretation** — What the results mean for the original question.

## Conclusions

Summarize:
- **Main takeaway** — The answer to your original question.
- **Implications** — How results could be used or acted upon.
- **Limitations** — Gaps in data, assumptions, or methods.
- **Next steps** — Further analyses, data to collect, or experiments to run.
